In [84]:
from pathlib import Path
import pickle
import numpy as np
import pandas as pd

In [85]:
labelmap = [
    'jump',
    'run',
    'throw',
    'kick',
    'bend',
    'dance',
    'clean_something',
    'squat',
    'punch',
    'crawl',
    'clap',
    'pick_up',
]

p_csv = Path('data/_filelists/babel/processed/filelist_babel_test_closed.txt')
df_csv = pd.read_csv(p_csv, delim_whitespace=True, header=None, names=['path', 'start', 'length', 'label'])
df_csv['path'] = df_csv['path'].map(
    lambda p_rel_video_stem: f'/data/datasets/babel_v1.0_release/renders/{p_rel_video_stem}.mp4')
df_csv['end'] = df_csv.apply(lambda row: f"{(row['start']+row['length']) / 30:.1f} s", axis=1)
df_csv['start'] = df_csv['start'].map(lambda idx_frame: f'{idx_frame / 30:.1f} s')
df_csv = df_csv[['path', 'start', 'end', 'label']]
df_csv

,path,start,end,label
0,/data/datasets/babel_v1.0_release/renders/val/...,2.1 s,3.2 s,11
1,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,2.2 s,1
2,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,1.5 s,11
3,/data/datasets/babel_v1.0_release/renders/val/...,21.5 s,23.2 s,4
4,/data/datasets/babel_v1.0_release/renders/val/...,0.7 s,2.1 s,7
...,...,...,...,...
447,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,26.6 s,1
448,/data/datasets/babel_v1.0_release/renders/val/...,99.4 s,102.3 s,11
449,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,25.8 s,1
450,/data/datasets/babel_v1.0_release/renders/val/...,2.6 s,3.5 s,4


In [86]:
p = 'work_dirs/train_output/cdar/03_simnreal/021_closed_k400_babel/01_tsm/021_dann_tmf/multiscale-tuning/dc_glx+cop-cross_lambda_x2/18088__closed_k2b-tsm-dann-tmf-dc_glx+cop-cross_lambda_x2/0/20230617-115700/best_pred.pkl'
with open(p, 'rb') as f:
    data = pickle.load(f)
data = np.array(data)
data.shape

(452, 12)

In [87]:
p_baseline = 'work_dirs/train_output/cdar/03_simnreal/021_closed_k400_babel/01_tsm/020_dann/multiscale/dc_glx+cop/11855__closed_k2b-tsm-dann-cross_dcx2_from_cop/0/20230522-230748/best_pred.pkl'
with open(p_baseline, 'rb') as f:
    data_baseline = pickle.load(f)
data_baseline = np.array(data_baseline)
data_baseline.shape

(452, 12)

In [88]:
RED = '\033[0;31m'
GREEN = '\033[0;32m'
NOCOLOR = '\033[0m' # No Color

preds = np.argmax(data, axis=1)
preds_baseline = np.argmax(data_baseline, axis=1)
df_csv['ours'] = preds
df_csv['ours_ox'] = np.where(preds == df_csv['label'], f'{GREEN}O{NOCOLOR}', f'{RED}X{NOCOLOR}')
df_csv['baseline'] = preds_baseline
df_csv['baseline_ox'] = np.where(preds_baseline == df_csv['label'], f'{GREEN}O{NOCOLOR}', f'{RED}X{NOCOLOR}')
df_csv

,path,start,end,label,ours,ours_ox,baseline,baseline_ox
0,/data/datasets/babel_v1.0_release/renders/val/...,2.1 s,3.2 s,11,11,[0;32mO[0m,9,[0;31mX[0m
1,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,2.2 s,1,1,[0;32mO[0m,1,[0;32mO[0m
2,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,1.5 s,11,11,[0;32mO[0m,11,[0;32mO[0m
3,/data/datasets/babel_v1.0_release/renders/val/...,21.5 s,23.2 s,4,11,[0;31mX[0m,3,[0;31mX[0m
4,/data/datasets/babel_v1.0_release/renders/val/...,0.7 s,2.1 s,7,9,[0;31mX[0m,3,[0;31mX[0m
...,...,...,...,...,...,...,...,...
447,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,26.6 s,1,1,[0;32mO[0m,1,[0;32mO[0m
448,/data/datasets/babel_v1.0_release/renders/val/...,99.4 s,102.3 s,11,11,[0;32mO[0m,11,[0;32mO[0m
449,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,25.8 s,1,1,[0;32mO[0m,1,[0;32mO[0m
450,/data/datasets/babel_v1.0_release/renders/val/...,2.6 s,3.5 s,4,9,[0;31mX[0m,9,[0;31mX[0m


In [89]:
df_csv = df_csv.loc[(preds == df_csv['label']) & (preds_baseline != df_csv['label'])]
df_csv

,path,start,end,label,ours,ours_ox,baseline,baseline_ox
0,/data/datasets/babel_v1.0_release/renders/val/...,2.1 s,3.2 s,11,11,[0;32mO[0m,9,[0;31mX[0m
23,/data/datasets/babel_v1.0_release/renders/val/...,2.8 s,5.6 s,6,6,[0;32mO[0m,3,[0;31mX[0m
37,/data/datasets/babel_v1.0_release/renders/val/...,1.2 s,7.2 s,8,8,[0;32mO[0m,2,[0;31mX[0m
45,/data/datasets/babel_v1.0_release/renders/val/...,2.9 s,4.4 s,11,11,[0;32mO[0m,6,[0;31mX[0m
56,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,10.6 s,9,9,[0;32mO[0m,6,[0;31mX[0m
61,/data/datasets/babel_v1.0_release/renders/val/...,15.7 s,17.1 s,7,7,[0;32mO[0m,3,[0;31mX[0m
72,/data/datasets/babel_v1.0_release/renders/val/...,3.6 s,40.9 s,5,5,[0;32mO[0m,2,[0;31mX[0m
76,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,33.4 s,5,5,[0;32mO[0m,2,[0;31mX[0m
87,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,4.1 s,10,10,[0;32mO[0m,8,[0;31mX[0m
96,/data/datasets/babel_v1.0_release/renders/val/...,2.0 s,7.9 s,9,9,[0;32mO[0m,3,[0;31mX[0m


In [90]:
df_csv['label'] = df_csv['label'].map(labelmap.__getitem__)
df_csv['ours'] = df_csv['ours'].map(labelmap.__getitem__)
df_csv['baseline'] = df_csv['baseline'].map(labelmap.__getitem__)
df_csv

/tmp/ipykernel_2218997/4066499689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv['label'] = df_csv['label'].map(labelmap.__getitem__)
/tmp/ipykernel_2218997/4066499689.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv['ours'] = df_csv['ours'].map(labelmap.__getitem__)
/tmp/ipykernel_2218997/4066499689.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

,path,start,end,label,ours,ours_ox,baseline,baseline_ox
0,/data/datasets/babel_v1.0_release/renders/val/...,2.1 s,3.2 s,pick_up,pick_up,[0;32mO[0m,crawl,[0;31mX[0m
23,/data/datasets/babel_v1.0_release/renders/val/...,2.8 s,5.6 s,clean_something,clean_something,[0;32mO[0m,kick,[0;31mX[0m
37,/data/datasets/babel_v1.0_release/renders/val/...,1.2 s,7.2 s,punch,punch,[0;32mO[0m,throw,[0;31mX[0m
45,/data/datasets/babel_v1.0_release/renders/val/...,2.9 s,4.4 s,pick_up,pick_up,[0;32mO[0m,clean_something,[0;31mX[0m
56,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,10.6 s,crawl,crawl,[0;32mO[0m,clean_something,[0;31mX[0m
61,/data/datasets/babel_v1.0_release/renders/val/...,15.7 s,17.1 s,squat,squat,[0;32mO[0m,kick,[0;31mX[0m
72,/data/datasets/babel_v1.0_release/renders/val/...,3.6 s,40.9 s,dance,dance,[0;32mO[0m,throw,[0;31mX[0m
76,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,33.4 s,dance,dance,[0;32mO[0m,throw,[0;31mX[0m
87,/data/datasets/babel_v1.0_release/renders/val/...,0.0 s,4.1 s,clap,clap,[0;32mO[0m,punch,[0;31mX[0m
96,/data/datasets/babel_v1.0_release/renders/val/...,2.0 s,7.9 s,crawl,crawl,[0;32mO[0m,kick,[0;31mX[0m


In [92]:
df_csv.reset_index().to_csv('analysis/result.csv', index=True, index_label='idx')